In [1]:
import pandas as pd
import numpy as np
import csv
import pickle
import geocoder
import time
from googlemaps import googlemaps 
gmaps = googlemaps.Client(tKey, timeout=10)

# Text matching

Read in the data

In [2]:
rraw = pd.read_csv("/Users/timothysweetser/Box Sync/Anna/village_kin/kin_locations_noexactmatch_clean.csv", dtype=str)
raw = rraw['village_kin']
raw = raw[~pd.isnull(raw)]
clean = pd.read_csv("/Users/timothysweetser/Box Sync/Anna/village_kin/clean_names.csv", dtype=str)
clean['state'][pd.isnull(clean['state'])] = 'NONE'
clean.head()

,name_short,state
0,AGRA,UTTAR PRADESH
1,AHMADABAD,GUJARAT
2,AHMADNAGAR,MAHARASHTRA
3,AJMER,RAJASTHAN
4,AKOLA,MAHARASHTRA


In [9]:
place = 'VASAI'
print place in raw
print place in matches.keys()
matches[place]

False
True


'Vasai, Maharashtra, India'

In [12]:
rraw[pd.isnull(rraw['village_kin'])]

,village_kin,state_respondent
10052,NaN,ORISSA


### File input/output

In [6]:
def save_obj(obj, name ):
    with open(name + '.pkl', 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

def load_obj(name ):
    with open(name + '.pkl', 'rb') as f:
        return pickle.load(f)

In [7]:
filename = "/Users/timothysweetser/box sync/anna/village_kin/matches_so_far"
matches = load_obj(filename)
print len(matches)

14757


In [86]:
k_no_response = 'no response from google API'
def geocode_api(place):
    # use google API directly, using API key
    try: 
        gc = gmaps.geocode(place + ', INDIA')
        time.sleep(2)
        if len(gc) > 0:
            return gc[0]
        else:
            return 'NA'
    except:
        return k_no_response

def geocode_ip(place):
    # use geocoder, which depends on IP address
    gc = geocoder.google(place + ', INDIA')
    time.sleep(2)
    if gc.address == None:
        return 'NA'
    else:
        return gc

In [29]:
print gc[0]['formatted_address']
print gc[0]['geometry']['location']

Limba, Maharashtra, India
{u'lat': 18.6313414, u'lng': 77.6335659}


In [7]:
for place in raw:
    if not (place in matches.keys()):
        # first pass done with a mix of ip and api calls

14370
14400
14430
14460
14490
14520
14550
14580
14610
14640
14670
14700
14730


In [36]:
print len(matches)
save_obj(matches, filename)

14757


In [13]:
print "There are %d total" % raw.nunique()
print "%d have been done " % len(matches)
print "There are %d left to do" %(raw.nunique() - len(matches))

There are 14757 total
14757 have been done 
There are 0 left to do


In [27]:
resultsDF = pd.DataFrame.from_dict(matches, orient='index').reset_index()
resultsDF.columns = ['village_kin', 'GMaps match']
resultsDF = resultsDF.merge(rraw, 'inner', 'village_kin')

print len(rraw.index)
print len(resultsDF.index) # dropped 1 NA
resultsDF = resultsDF[['village_kin', 'state_respondent', 'GMaps match']]
print resultsDF.head()
no_match = resultsDF['GMaps match'] == 'NA'
print "%d had no match" % sum(no_match)
import random
resultsDF.ix[no_match].tail(10)

15390
15389
     village_kin state_respondent                             GMaps match
0       CHAMARAM      WEST BENGAL                                      NA
1    PUDARIPATTY       TAMIL NADU                                      NA
2  CHERUGOLPIZHA           KERALA            Cherukolpuzha, Kerala, India
3         PIPREE   MADHYA PRADESH     Pipree, Uttar Pradesh 272002, India
4      POONGNOOR   ANDHRA PRADESH  Punganur, Andhra Pradesh 517247, India
6780 had no match


,village_kin,state_respondent,GMaps match
15371,MANEPALLI,ANDHRA PRADESH,NA
15374,KUTHALAM,TAMIL NADU,NA
15375,TANWADI,MAHARASHTRA,NA
15378,SELA,RAJASTHAN,NA
15379,SOMTEPALLI,ANDHRA PRADESH,NA
15382,GHANERAM,RAJASTHAN,NA
15383,TAMILNADU,KERALA,NA
15385,SARSHA,HARYANA,NA
15386,SODAKO,HARYANA,NA
15388,MALLAGODU,TAMIL NADU,NA


In [35]:
# write to file
filename = "/Users/timothysweetser/Box Sync/Anna/village_kin/google maps suggestions.txt"
resultsDF.to_csv(filename, sep="\t", index=False, encoding='utf-8')

### Second pass - add lat/long

In [166]:
#take2 = {}
for place, val in matches.iteritems():
    if place not in take2.keys():
        take2[place] = geocode_api(place)

save_obj(take2, "/Users/timothysweetser/box sync/anna/village_kin/matches_take2")

In [167]:
counter = 0L
for val in take2.values():
    if val == 'NA':
        counter += 1
print counter

3643


In [168]:
counter = 0L
for val in take2.values():
    if val == k_no_response:
        counter += 1
print counter

42


In [169]:
len(take2)

14757

In [94]:
# try 'no responses' again
for place, val in take2.iteritems():
    if val == 'no response from google API':
        #take2[place] = geocode_api(place)
        print place

RANGANAYAKANH KOPPALO
HULKERE JKOPLU
KAUARVRAVHI
KAMRIREDDY WARIPLLU
MADEKE RANHALLD
MALLANAM PATTEY
SHANGH RAHDIH
MALLLANAM PATTY
RANGAOANDEB
AMARSIN GHWRA
NADLNIDHAURA
BHATVA POTIEE
PEDDATALA KUNTLA
SAREMRURUR
GHUMUNVARHI
PADOALIN GAVAVASA
BANDIMA KANAKOPALU
DHADHUUAMU PATTI
NENGEGOUDA KOPPLU
PERAMBOTLA PELAM
VALKAM- OINNI
PAPPALINSARHM
MADAKERANALLLI
FRAND NAGAR BORGHDA
THVVARAKVRIGHI
PERAMROTLA PLEM
BILLASWAR RASUI
KAMREDY WARI PALLE
PAPPAUNGAVILASA
KDAIKA PALIIA
SARHUNVA
SAPBUVANUDAI
PERAMBITLA PACEM
PERAMBOTLA PRLEM
PEDDAYAKLLA KUNTLA
KEBNFARARA                   


In [179]:
save_obj(take2, "/Users/timothysweetser/box sync/anna/village_kin/matches_take2")

In [171]:
take2[take2.keys()[15]]['formatted_address']

u'ULM Amangani, Garhi Bolni Road, Shabajpur Khalsa, Haryana 123110, India'

In [173]:
print take2[take2.keys()[15]]['geometry']['location']['lat']
print take2[take2.keys()[15]]['geometry']['location']['lng']

28.1573853
76.6333508


In [174]:
take2_select = {}
for place, val in take2.iteritems():
    if type(val) == dict:
        take2_select[place] = {'GMaps match':val['formatted_address'], \
            'latitude':val['geometry']['location']['lat'], 'longitude':val['geometry']['location']['lng']}
    else:
        take2_select[place] = {'GMaps match': 'NA', 'latitude':np.nan, 'longitude':np.nan}
        
take2_select = pd.DataFrame.from_dict(take2_select, orient='index').reset_index()
take2_select.rename(columns={'index':'village_kin'}, inplace=True)

In [175]:
# merge in state_respondent
take2_select = take2_select.merge(rraw, 'inner', 'village_kin')

In [176]:
take2_select = take2_select[['village_kin', 'state_respondent', 'GMaps match', 'latitude', 'longitude']]
take2_select.head()

,village_kin,state_respondent,GMaps match,latitude,longitude
0,A.KUNTLA,ANDHRA PRADESH,"Khoontla, Rajasthan 303326, India",26.961655,76.545933
1,AADAR SHNAGAR,MADHYA PRADESH,"Aadhar Society, Aadhar Society, Satyam Park So...",22.297099,73.228433
2,AADARI,GUJARAT,"Adari, Uttar Pradesh 275102, India",25.979569,83.611819
3,AADI,HARYANA,"Adi, Karnataka 591215, India",16.496157,74.372676
4,AADRI,GUJARAT,"Adri, Gujarat 362255, India",20.964116,70.289546


In [165]:
take2_select.to_csv('/Users/timothysweetser/Box Sync/Anna/village_kin/google maps suggestions lat long.txt', \
                    sep='\t', index=False, encoding='utf-8')

In [177]:
len(take2_select.index)

15389

In [178]:
take2_select.head()

,village_kin,state_respondent,GMaps match,latitude,longitude
0,A.KUNTLA,ANDHRA PRADESH,"Khoontla, Rajasthan 303326, India",26.961655,76.545933
1,AADAR SHNAGAR,MADHYA PRADESH,"Aadhar Society, Aadhar Society, Satyam Park So...",22.297099,73.228433
2,AADARI,GUJARAT,"Adari, Uttar Pradesh 275102, India",25.979569,83.611819
3,AADI,HARYANA,"Adi, Karnataka 591215, India",16.496157,74.372676
4,AADRI,GUJARAT,"Adri, Gujarat 362255, India",20.964116,70.289546
